In [1]:
#keras Functional Model

from keras.layers import Input , Dense
from keras.models import Model

In [2]:
inputs = Input(shape=(784 , ))

In [3]:
x = Dense(64 , activation='relu')(inputs)
x = Dense(64 , activation='relu')(x)

predictions = Dense(10 , activation='softmax')(x)

In [4]:
model = Model(inputs=inputs , outputs=predictions)

model.compile(optimizer='rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

model.fit(data , labels)

NameError: name 'data' is not defined

In [6]:
#模型类似函数 可以作为运算的操作
will_input_data = Input(shape=(784 , ))
y = model(will_input_data)
#模型可以被作为层一样
#调用一个模型时 即重用结构 也重用权重

In [7]:
from keras.layers import TimeDistributed

input_sequences = Input(shape=(20,784))

processed_sequences = TimeDistributed(model)(input_sequences)

In [ ]:
#http://keras-cn.readthedocs.io/en/latest/getting_started/functional_API/